In [53]:
import h5pyd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.spatial import cKDTree

END_POINT = 'https://developer.nrel.gov/api/hsds/'
#API_TOKEN = '3K3JQbjZmWctY0xmIfSYvYgtIcM3CN0cb1Y2w9bf' 
#API_TOKEN = 'TwI2au3SB4xWmXImLmswNQaRDE7lxNlK0AFlhLMj' #wangyao0317
#API_TOKEN = 'YRog0tZ7eozEnjDmxQyTlWv9WSEYDnJdXhplHuWI' #yw2690
API_TOKEN = 'cpYoxAkgLjrWrasC3RcXh0guItVH6TIKxElcufcP' #brightpower
# for security reason, personal token is hidden
# this token for demo only, go to https://developer.nrel.gov/signup/ for actual token



In [6]:
# yr_2017 = "/nrel/nsrdb/nsrdb_2017.h5"

# #Find solar irradiance difference at different city
# #Example1:NYC and Miami
# #Example2:San Diego, Seattle

# f_17 = h5pyd.File(yr_2017, 'r', endpoint=END_POINT, api_key=API_TOKEN)
# ghi_17 = f_17['ghi']
# #more factors to consider
# #temp_17 = f_17['air_temperature']
# #rain_17 = f_17['total_precipitable_water']
# #windspeed_17 = f_17['wind_speed']
 
# meta = pd.DataFrame(f_17['meta'][...])
# #meta_state = meta['state']

In [7]:
# meta_US = meta.loc[meta['country'] == b'United States'] # Note .h5 saves strings as bit-strings
# df_state = meta_US['state'].to_frame(name = 'state')


In [8]:
# #ghi = pd.DataFrame(f_17['ghi'][...])
# #ghi_state = ghi.sum(axis = 0, skipna = True)
# time_index = pd.to_datetime(f_17['time_index'][...].astype(str))


In [15]:
# timestep = np.where(time_index == '2017-06-23 00:00:00')[0][0]
# timestep

8304

In [1]:
# yr_2017 = "/nrel/nsrdb/nsrdb_2017.h5"

# #Find solar irradiance difference at different city
# #Example1:NYC and Miami
# #Example2:San Diego, Seattle

# f_17 = h5pyd.File(yr_2017, 'r', endpoint=END_POINT, api_key=API_TOKEN)
# ghi_yr = f_17['ghi']
# temp_yr = f_17['air_temperature']
# rain_yr = f_17['total_precipitable_water']
# windspeed_yr = f_17['wind_speed']
 
# pd.DataFrame(dfs[timestep] for dfs in (ghi_yr,temp_yr,rain_yr,windspeed_yr))
 

# df_ghi.columns = ['ghi','air_temperature','total_precipitable_water','wind_speed']


In [2]:
# df_ghi_state = pd.merge(df_ghi, df_state,left_index=True, right_index = True)
# df_ghi_state.groupby('state').mean()

In [15]:
#year = "/nrel/nsrdb/nsrdb_2017.h5"

def get_f(year):
    yr = '/nrel/nsrdb/nsrdb_'+str(year)+'.h5'
    f_yr = h5pyd.File(yr, 'r', endpoint=END_POINT, api_key=API_TOKEN)
    return f_yr



In [ ]:
import dill
dill.dump(get_f(2017), open('f_2017.csv', 'wb'))    


Load file from dill

In [2]:
import dill
f_17 = dill.load(open('f_2017.csv','rb'))

In [36]:
f_17

<HDF5 file "nsrdb_2017.h5" (mode r)>

In [18]:
#NREL data currently spans 1998 to 2017
#Each API can only access 15 times per day
#First API accessed from 1998-2012
#Second API accessed from 2013-2017

for year in range(2013,2018):
    dill.dump(get_f(year), open('f_'+str(year)+'.csv', 'wb'))    

In [33]:
import dill
#load file data for 20 yrs

full_data={}

for year in range(1998,2018):
   full_data[year] = dill.load(open('f_'+str(year)+'.csv','rb'))

In [13]:
# def get_ghi(year,f_yr):
#     ghi_yr = f_yr['ghi']
#     meta = pd.DataFrame(f_yr['meta'][...])
#     meta_US = meta.loc[meta['country'] == b'United States'] # Note .h5 saves strings as bit-strings
#     df_state = meta_US['state'].to_frame(name = 'state')
#     time_index = pd.to_datetime(f_yr['time_index'][...].astype(str))
    
#     fix_date = str(year)+'-06-23 00:00:00'
#     timestep = np.where(time_index == fix_date)[0][0]
    
#     df_ghi = pd.DataFrame(f_yr['ghi'][timestep])
#     df_ghi.columns = ['ghi']
#     df_ghi_state = pd.merge(df_ghi, df_state,left_index=True, right_index = True)
#     state_ghi = df_ghi_state.groupby('state').mean()
#     return state_ghi
    
# get_ghi(2017,f_17)
# dill.dump(get_ghi(2017,f_17), open('ghi_2017.csv', 'wb'))    
# f_17 = dill.load(open('f_2017.csv','rb'))

In [52]:
full_data[2017]

<HDF5 file "nsrdb_2017.h5" (mode r)>

In [38]:
def get_attributes(year,f_yr):

    ghi_yr = f_yr['ghi']    
    temp_yr = f_yr['air_temperature']
    rain_yr = f_yr['total_precipitable_water']
    windspeed_yr = f_yr['wind_speed']
    
    meta = pd.DataFrame(f_yr['meta'][...])
    meta_US = meta.loc[meta['country'] == b'United States'] # Note .h5 saves strings as bit-strings
    df_state = meta_US['state'].to_frame(name = 'state')
    time_index = pd.to_datetime(f_yr['time_index'][...].astype(str))
    
    fix_date = str(year)+'-06-23 00:00:00'
    timestep = np.where(time_index == fix_date)[0][0]
    

    df_attr = pd.DataFrame(ghi_yr[timestep])
    df_attr.columns = ['ghi']
    df_attr['temp'] = pd.DataFrame(temp_yr[timestep])
    df_attr['rain'] = pd.DataFrame(rain_yr[timestep])
    df_attr['windspeed'] = pd.DataFrame(windspeed_yr[timestep])
    
    df_ghi_state = pd.merge(df_attr, df_state,left_index=True, right_index = True)
    state_ghi = df_ghi_state.groupby('state').mean()
    return state_ghi

get_attributes(2017,full_data[2017])

OSError: Error retrieving data: None

In [54]:
result_full_data={}

In [55]:


#for year in range(1998,2010):
year=1998
result_full_data[year] = get_attributes(year,full_data[year])



OSError: Error retrieving data: None

In [56]:
#change to another API
for year in range(2010,2018):
   result_full_data[year] = get_attributes(year,full_data[year])

KeyboardInterrupt: 

In [50]:
result_full_data

{}

In [11]:
dill.dump(nrel_17, open('result_17.csv','wb'))

In [41]:
result_17 = dill.load(open('result_17.csv','rb'))

In [42]:


import matplotlib
import seaborn as sns

result_17

,ghi,temp,rain,windspeed
state,,,,
b'Alabama',69.439400,24.803498,5378.932142,7.102841
b'Alaska',493.703327,12.049359,1734.823831,20.484018
b'Arizona',444.054861,37.008627,1641.401947,56.256443
b'Arkansas',108.703920,25.623991,6253.498654,11.151083
b'California',579.746328,34.785506,1716.497539,27.186428
b'Colorado',254.097771,26.559943,1889.411543,32.915429
b'Connecticut',27.524533,20.128505,4221.529206,2.688084
b'Delaware',23.321429,25.425595,4640.505952,17.785714
b'District of Columbia',29.272727,26.727273,4154.181818,16.000000
